### Sentiment analysis on reviews data
Kei Sato

ML310B - Advanced Machine Learning

March 25, 2019


We will be using reviews data to develop a sentiment analyzer, such that given a document, the model can predict if the review is positive (sentiment = 1) or negative (sentiment = 0)


In [22]:
# Load the data...
import pandas as pd

data = pd.read_csv('data/Reviews.csv')

print("Number of positive and negative review", '\n', data["sentiment"].value_counts())
data.head()

Number of positive and negative review 
 1    25000
0    25000
Name: sentiment, dtype: int64


,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [2]:

x_train, x_test, y_train, y_test = train_test_split(data["review"], data["sentiment"], test_size=0.3, random_state=42)

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Taken Lesson 8 HW assignment
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

replace_re_by_space = re.compile('[/(){}\[\]\|@,;]')
delete_re_symbols = re.compile('[^0-9a-z #+_]')
stop_words =  set(stopwords.words('english'))

def combine_tokened_words(tokened_words):
    length_of_string=len(tokened_words)
    text_new=""
    for w in tokened_words:
        if w!=tokened_words[length_of_string-1]:
             text_new=text_new+w+" " # when w is not the last word so separate by whitespace
        else:
            text_new=text_new+w
    return text_new

# Returns tokenized words
def text_processing(text):
    text = text.lower()
    text = text.replace("<br />", '')
    text = re.sub(replace_re_by_space.pattern, ' ', text)
    text = re.sub(delete_re_symbols.pattern, '', text)
    token_word = word_tokenize(text)
    
    # filtered_sentence contain all words that are not in stopwords dictionary
    filtered_sentence = [w for w in token_word if not w in stop_words]
    return filtered_sentence

def text_lemmatization(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    text = list(map(lambda word: wordnet_lemmatizer.lemmatize(word), text))
    return text
# text_lemmatization(text_processing(x_train.values[0]))


# Update to pipeline???
# get test and train data
# run through tfidif vectorizer
# go through logistic regression (???) w/ cross validation
# output accuracy
def run_model(data):
    data["review"] = data["review"].apply(lambda text: combine_tokened_words(
        text_lemmatization(text_processing(text))))   
    x_train, x_test, y_train, y_test = train_test_split(
        data["review"],
        data["sentiment"],
        test_size=0.3,
        random_state=42
    )
    tfid_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9).fit(x_train)
    x_train = tfid_vectorizer.transform(x_train)
    x_test = tfid_vectorizer.transform(x_test)

    model = LogisticRegression().fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("accuracy", accuracy_score(y_test, y_pred))
    
    
run_model(data.copy(deep=True))



accuracy 0.83
